### Preparation

In [ ]:
cd("../../src/")
include("modeling/correlation_lib.jl")
include("modeling/util.jl")
using SQLite
using CSV
using DataFrames: DataFrame, Missing
using Statistics, LinearAlgebra, Logging, Random
using Gen

cd("../data/rent_data/")

db = SQLite.DB()
rent_table = CSV.File("rent_data.csv") |> SQLite.load!(db, "rent_table")
k_score_table = CSV.File("rent_data_1000.csv") |> SQLite.load!(db, "k_score_table")

(observation_df, _, _, _) = parse_data(db)

attrs = Dict(:state => :dirty_state,
            :city => :dirty_city,
            :zip => :dirty_zip,
            :rent => :rent);

# Rent Model
Implements a generative model that 
- reads a dataset (observation) of rent data,
- incorperates external domain knowledge (e.g. zips are uniquely assigned to cities), 
- samples new rent data based on the statistical information extracted from observation (occurrence statistics, correlation between columns etc.) and
- evaluates the probability of given data entry being sampled by this model.

#### Columns of the observation dataset
- __State__: Names of German states
- __City__: Names of German cities
- __Zip__: We assume for now that one city has only one zip code
- __Rent price__

#### Types of error within observation:
- __Typos__: State, city and zip could be misspelled
- __Conflict__: State, city and zip could be conflicting
- __Outlier__: Rent price could be abnormally high/low

#### Assumptions
- We have no idea about which cities and states exist in Germany
- We cannot check which cities belong to which state
- The observation dataset is "mostly" correct, that is, most values and combinations are trustworthy
- We know the fundamental connections between state, city and zip
- We assume the rent prices within the same city are similar to some extent

#### Things we can do with this model:
- Sample fake data
- Detect outlier: probability evaluation for arbitrary combinations of column values
- Correct data: filling out missing data with most probable values, give suggestions for outliers -> Inference

In [ ]:
observation_df

### 1. Defining sampling process
We think about how the dataset is generated in the first place, and define the sampling order for the columns.
1. __State__: the more often a state value appears in our observation, the more probable it is to be clean, thus 

$P(State="Bayern")\propto n_{State="Bayern"}$. 

The sampler samples ```sampled_state```.
2. __City__: While the same story applies to city, we think that cities that co-occurs with ```sampled_state``` are probably clean, and the number of co-occurrence matters. Thus 

$P(City="Muenchen"|State=sampled\_state)\propto n_{State=sampled\_state\land City="Muenchen"} \cdot n_{City="Muenchen"}$.

The sampler samples ```sampled_city```.
3. __Zip__: The total occurrence of zip codes and co-occurrence with ```sampled_city``` and ```sampled_state``` are considered.
4. __Rent__: We take the set of observed data entries with```sampled_state```, ```sampled_city``` and ```sampled_zip```, calculate mean and standard deviation of their rent prices, and sample from the according normal distribution.


In [ ]:
@gen function rent_model(realization_df)
    @info "-----------------"
    state = @trace(occurrence(realization_df,
                            :dirty_state), :state)
    @info "Sampled state: $state"

    city = @trace(co_occurrence(realization_df,
                            [:dirty_state,],
                            :dirty_city,
                            [String(state)],
                            true), :city)
    @info "Sampled city: $city"

    zip = @trace(co_occurrence(realization_df,
                                    [:dirty_state, :dirty_city],
                                    :dirty_zip,
                                    [String(state), String(city)],
                                    true), :zip)
    @info "Sampled zip: $zip"

    rent = @trace(numerical_functional_dependency(realization_df,
                                                        [:dirty_state, :dirty_city, :dirty_zip],
                                                        :rent,
                                                        [String(state), String(city), String(zip)],
                                                        true,
                                                        true), :rent)
    new_rent = @trace(numerical_co_occurrence(realization_df,
            [:dirty_state, :dirty_city, :dirty_zip],
            [],
            (:rent, 1.),
            [String(state), String(city), String(zip)],
            true,
            false), :new_rent)
    @info "Sampled rent: $rent and $new_rent"
    end;

### 2. Sampling synthetic data
By letting the model run, it samples new entries based on the knowledge it discovers from the observation.

In [ ]:
#disable_logging(LogLevel(10))

trace = Gen.simulate(rent_model, (observation_df, ))
score = get_score(trace)
println("$(trace[:state => :realization]) \n$(trace[:city => :realization]) \n$(trace[:zip => :realization]) \n$(trace[:rent => :realization])")
println("Log-likelihood: $score")

### 3. Evaluating probability for given data entry
For given entry $x$, the model returns the log-likelihood of the joint probability:

$L(x) = log(P(x|rent\_model)) =$

$log(P(State=x.state)\cdot P(City=x.city|State=x.state)\cdot P(Zip=x.zip|State=x.state \land City=x.city) \cdot P(Rent=x.rent|City=x.city \land Zip=x.zip))$

In [ ]:
constraints = Gen.choicemap()
constraints[:state => :realization] = Symbol("Bayern")
constraints[:city => :realization] = Symbol("Muenchen")
constraints[:zip => :realization] = Symbol("80331")
constraints[:rent => :realization] = 11.73

(trace, weight) = Gen.generate(rent_model, (observation_df, ), constraints)
println("$(trace[:state => :realization])
$(trace[:city => :realization])
$(trace[:zip => :realization])
$(trace[:rent => :realization])")
score = get_score(trace)
println("Log-likelihood: $score\n")

#### For the observation data, the model can output e.g. $k$ entries that are the most improbable:

In [ ]:
k_score_df = SQLite.Query(db, "SELECT dirty_state, dirty_city, dirty_zip, rent FROM k_score_table") |> DataFrame
k_score_df

In [ ]:
# observation_df_short = observation_df[1:2000, :]
(k_traces, k_scores) = k_most_improbable(10, k_score_df, attrs, rent_model)

for trace in k_traces
    println("--------------------")
    println("$(trace[:state => :realization])\n$(trace[:city => :realization])\n$(trace[:zip => :realization])\n$(trace[:rent => :realization])")
    score = get_score(trace)
    println("Log-likelihood: $score\n")
end

#### Some facts about the evaluation dataset:
- $n_{Kaufbeuren} = n_{Passau} = 1$
- $n_{Hof} = n_{Bayreuth} = 2$

### 4. Generating repair suggestion for outliers
For entries with missing column values, the model uses e.g. importance resampling to find possible values that maximize the joint probability.

In [ ]:
function do_inference(model, args, constraints, num_iter)
    (trace, lml_est) = Gen.importance_resampling(model, args, constraints, num_iter)
    return (trace, lml_est)
end

constraints = Gen.choicemap()
#constraints[:state => :realization] = Symbol("Bayern")
constraints[:city => :realization] = Symbol("Muenchen")
constraints[:zip => :realization] = Symbol("80331")
constraints[:rent => :realization] = 11.73

(trace, _) = do_inference(rent_model, (observation_df, ), constraints, 50)
println("$(trace[:state => :realization]) \n$(trace[:city => :realization]) \n$(trace[:zip => :realization]) \n$(trace[:rent => :realization])")
score = get_score(trace)
println("Log-likelihood: $score")

==================================================================
### What is achieved?
1. A generative model for outlier detection and repair suggestion. It can be both supervised and completely unsupervised.
2. A programming paradigm that can be applied to other database problems from other domains. 

### General problems
1. The number of occurrence is assumed to have strong meaning for correctness. What if this is not the case?
   
   $\to$ More external knowledge is required (e.g. list of cities)
   
   
2. For each categorical column, a categorical distribution is estalished, where each categorical value has a certain probability that is proportional to e.g. the number of co-occurrence with values from another categorical column. Such cross-column relationships are stated explicitly. What if they are unknown or imprecise?
    
   $\to$ Learn vector representations for categorical values, where the distance between their vector representations implies the "similarity" between them


3. The model implements external knowledge and statistical findings in a "hard" fashion, i.e. there are no trainable parameters that can improve performance over time.


4. Performance comparison between such generative models and conventional methods has to be conducted.

### Other problems    
1. If e.g. "Bayrn", "Münhen", "80331", "10€" appearred only once in the dataset, what is the distribution of rent?
   
   $\to$ Find better way of modeling the distribution of numerical columns


2. Repair suggestion requires better inference algorithm to avoid combination explosion.


3. Other performance issues that are related to implementation.